In [1]:
## Required Packages
from termcolor import colored
import math
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from datetime import datetime
import pickle as pkl
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim
import tqdm
import time
import transformers
from sklearn.metrics import roc_auc_score  
from sklearn.metrics import roc_curve 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
%matplotlib inline
use_cuda = torch.cuda.is_available()
import transformers
from transformers import BertForSequenceClassification

In [2]:
train_f=pkl.load( open('./fineTuneData/CI_train.pkl', 'rb'), encoding='bytes')
valid_f=pkl.load( open('./fineTuneData/CI_valid.pkl', 'rb'), encoding='bytes')
test_f=pkl.load( open('./fineTuneData/CI_test.pkl', 'rb'), encoding='bytes')
test_f2=pkl.load( open('./fineTuneData/CI_test.pkl', 'rb'), encoding='bytes')

In [3]:
print (len(train_f),len(test_f),len(valid_f))

2124 591 236


### Data preprocessing

In [4]:
### Below are key functions for  Data prepartion ,formating input data into features, and model defintion 

class PaddingInputExample(object):
  """Fake example so the num input examples is a multiple of the batch size.

  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.

  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    self.is_real_example = is_real_example
    

    
def convert_EHRexamples_to_features(examples,max_seq_length):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    features = []
    for (ex_index, example) in enumerate(examples):
        feature = convert_singleEHR_example(ex_index, example, max_seq_length)
        features.append(feature)
    return features

### This is the EHR version

def convert_singleEHR_example(ex_index, example, max_seq_length):
    if isinstance(example, PaddingInputExample):
        return InputFeatures(
        input_ids=[0] * max_seq_length,
        input_mask=[0] * max_seq_length,
        segment_ids=[0] * max_seq_length,
        label_id=0,
        is_real_example=False)
    
    input_ids=example[2]
    segment_ids=example[3]
    label_id=example[1]
    

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
    input_mask = [1] * len(input_ids)

   
  # LR 5/13 Left Truncate longer sequence 
    while len(input_ids) > max_seq_length:
        # input_ids= input_ids[-max_seq_length:] 
        # input_mask= input_mask[-max_seq_length:]
        # segment_ids= segment_ids[-max_seq_length:]
        input_ids= input_ids[:max_seq_length] 
        input_mask= input_mask[:max_seq_length]
        segment_ids= segment_ids[:max_seq_length]
    
    
  # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

  
    feature =[input_ids,input_mask,segment_ids,label_id,True]
    return feature


### Data Loader

In [5]:
class BERTdataEHR(Dataset):
    def __init__(self, Features):
           
        self.data= Features
  
                                     
    def __getitem__(self, idx, seeDescription = False):

        sample = self.data[idx]
   
        return sample

    def __len__(self):
        return len(self.data)     

         
#customized parts for EHRdataloader
def my_collate(batch):
        all_input_ids = []
        all_input_mask = []
        all_segment_ids = []
        all_label_ids = []

        for feature in batch:
            all_input_ids.append(feature[0])
            all_input_mask.append(feature[1])
            all_segment_ids.append(feature[2])
            all_label_ids.append(feature[3])
        return [all_input_ids, all_input_mask,all_segment_ids,all_label_ids]
            

class BERTdataEHRloader(DataLoader):
    def __init__(self, dataset, batch_size=128, shuffle=False, sampler=None, batch_sampler=None,
                 num_workers=0, collate_fn=my_collate, pin_memory=False, drop_last=False,
                 timeout=0, worker_init_fn=None):
        DataLoader.__init__(self, dataset, batch_size=batch_size, shuffle=False, sampler=None, batch_sampler=None,
                 num_workers=0, collate_fn=my_collate, pin_memory=False, drop_last=False,
                 timeout=0, worker_init_fn=None)
        self.collate_fn = collate_fn

 

In [6]:
# use_cuda = False

### Model Definition LR

In [7]:
class EHR_BERT_LR(nn.Module):
    def __init__(self, input_size,embed_dim, hidden_size, n_layers=1,dropout_r=0.1,cell_type='LSTM',bi=False ,time=False, preTrainEmb=''):
        super(EHR_BERT_LR, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.dropout_r = dropout_r
        self.cell_type = cell_type
        self.preTrainEmb=preTrainEmb
        self.time=time
        
        if bi: self.bi=2 
        else: self.bi=1
        
        self.PreBERTmodel=BertForSequenceClassification.from_pretrained("./pytorch_model/", ignore_mismatched_sizes=True)
        if use_cuda:
           self.PreBERTmodel.cuda()
        self.input_size=self.PreBERTmodel.bert.config.vocab_size
        self.in_size= self.PreBERTmodel.bert.config.hidden_size
       
        self.dropout = nn.Dropout(p=self.dropout_r)
        self.out = nn.Linear(self.in_size,1)
        self.sigmoid = nn.Sigmoid()
        self.softmax=nn.Softmax()
        if use_cuda:
            self.lnt_typ=torch.cuda.LongTensor
            self.flt_typ=torch.cuda.FloatTensor
        else: 
            self.lnt_typ=torch.LongTensor
            self.flt_typ=torch.FloatTensor

    def forward(self, sequence):
        # print('call model')
        token_t=torch.from_numpy(np.asarray(sequence[0],dtype=int)).type(self.lnt_typ)
        seg_t=torch.from_numpy(np.asarray(sequence[2],dtype=int)).type(self.lnt_typ)
        Label_t=torch.from_numpy(np.asarray(sequence[3],dtype=int)).type(self.lnt_typ)
        # print(token_t)
        # print(seg_t.size())
        Bert_out=self.PreBERTmodel.bert(input_ids=token_t
                                    , attention_mask=torch.from_numpy(np.asarray(sequence[1],dtype=int)).type(self.lnt_typ)
                                    , token_type_ids=seg_t)
        output=self.sigmoid(self.out(Bert_out[1]))
        # print(len(Bert_out))
        # print(output.squeeze().size())
        # print('=============================')
        return output.squeeze(),Label_t.type(self.flt_typ)

In [8]:
def timeSince(since):
   now = time.time()
   s = now - since
   m = math.floor(s / 60)
   s -= m * 60
   return '%dm %ds' % (m, s)

def trainsample(sample, model, optimizer, criterion = nn.BCELoss()): 
    # print('here2')
    model.train() 
    model.zero_grad()
    output,label_tensor = model(sample)  
    # print('hereeeee') 
    # print(output)
    # print(label_tensor.size())
    # print(label_tensor.unsqueeze(1).size())
    loss = criterion(output, label_tensor)    
    loss.backward()   
    optimizer.step()
    return output, loss.item()


#train with loaders

def trainbatches(mbs_list, model, optimizer, shuffle = True):
    # print('here1')
    current_loss = 0
    all_losses =[]
    plot_every = 5
    n_iter = 0 
    if shuffle: 
        random.shuffle(mbs_list)
    for i,batch in enumerate(mbs_list):
        # print(type(batch))
        # print(len(batch[0]), len(batch[1]), len(batch[2]), len(batch[3]))
        output, loss = trainsample(batch, model, optimizer, criterion=nn.BCELoss())
        current_loss += loss
        n_iter +=1
    
        if n_iter % plot_every == 0:
            all_losses.append(current_loss/plot_every)
            current_loss = 0    
    return current_loss, all_losses 


def calculate_auc(model, mbs_list, shuffle = True): 
    model.eval() 
    y_real =[]
    y_hat= []
    if shuffle: 
        random.shuffle(mbs_list)
    for i,batch in enumerate(mbs_list):
        output,label_tensor = model(batch)
        y_hat.extend(output.cpu().data.view(-1).numpy())  
        y_real.extend(label_tensor.cpu().data.view(-1).numpy())       
    auc = roc_auc_score(y_real, y_hat)
    return auc, y_real, y_hat 

    
#define the final epochs running, use the different names

def epochs_run(epochs, train, valid, test1, test2, model, optimizer, shuffle=True, patience=20, output_dir='./models/', model_prefix='MED_BERT_', model_customed='CI_'):  
    bestValidAuc = 0.0
    bestTestAuc1 = 0.0
    bestTestAuc2 = 0.0
    bestValidEpoch = 0
    #header = 'BestValidAUC|TestAUC|atEpoch'
    #logFile = output_dir + model_prefix + model_customed +'EHRmodel.log'
    #print2file(header, logFile)
    #writer = SummaryWriter(output_dir+'/tsb_runs/') ## LR added 9/27 for tensorboard integration
    for ep in range(epochs):
        print (ep)
        start = time.time()
        current_loss, train_loss = trainbatches(mbs_list=train, model=model, optimizer=optimizer)
        train_time = timeSince(start)
        #epoch_loss.append(train_loss)
        avg_loss = np.mean(train_loss)
        #writer.add_scalar('Loss/train', avg_loss, ep) ## LR added 9/27 
        valid_start = time.time()
        train_auc, _, _ = calculate_auc(model = model, mbs_list = train, shuffle = shuffle)
        valid_auc, _, _ = calculate_auc(model = model, mbs_list = valid, shuffle = shuffle)
        valid_time = timeSince(valid_start)
        #writer.add_scalar('train_auc', train_auc, ep) ## LR added 9/27 
        #writer.add_scalar('valid_auc', valid_auc, ep) ## LR added 9/27 
        print(colored('\n Epoch (%s): Train_auc (%s), Valid_auc (%s) ,Training Average_loss (%s), Train_time (%s), Eval_time (%s)'%(ep, train_auc, valid_auc , avg_loss,train_time, valid_time), 'green'))
        if valid_auc > bestValidAuc: 
              bestValidAuc = valid_auc
              bestValidEpoch = ep
              best_model= model 
              bestTrainAuc = train_auc  
              if test:      
                      testeval_start = time.time()
                      bestTestAuc1, _, _ = calculate_auc(model = best_model, mbs_list = test1,  shuffle = shuffle) 
                      bestTestAuc2, _, _ = calculate_auc(model = best_model, mbs_list = test2,  shuffle = shuffle) 

                      #writer.add_scalar('test_auc', valid_auc, ep) ## LR added 9/27 
                      print(colored('\n Test_AUC1 (%s) ,Test_AUC2 (%s) , Test_eval_time (%s) '%(bestTestAuc1,bestTestAuc2, timeSince(testeval_start)), 'yellow')) 
                      #print(best_model,model) ## to verify that the hyperparameters already impacting the model definition
                      #print(optimizer)
        if ep - bestValidEpoch > patience:
              break

    # model.PreBERTmodel.save_pretrained(save_directory='./fineTuneModels/fixed_last/') #,state_dict=model.state_dict()
    # #writer.close()
    # if not os.path.exists(output_dir):
    #    os.makedirs(output_dir)
    # ##save model & parameters
    # torch.save(best_model, output_dir + model_prefix + model_customed + 'EHRmodel.pth')
    # torch.save(best_model.state_dict(), output_dir + model_prefix + model_customed + 'EHRmodel.st')

    if test:
        print(colored('BestValidAuc %f has a TestAuc of %f at epoch %d ' % (bestValidAuc, bestTestAuc1, bestValidEpoch),'green'))
        return bestTrainAuc,bestValidAuc, bestTestAuc1, bestTestAuc2, bestValidEpoch
    else: 
        print(colored('BestValidAuc %f at epoch %d ' % (bestValidAuc,  bestValidEpoch),'green'))
        print('No Test Accuracy')
    
    print(colored('Details see ../models/%sEHRmodel.log' %(model_prefix + model_customed),'green'))
        


In [9]:
MAX_SEQ_LENGTH = 512
BATCH_SIZE = 64
LEARNING_RATE = 1e-5
bert_config_file= "config.json"

results=[]

#### Data Preparation
train_features = convert_EHRexamples_to_features(train_f, MAX_SEQ_LENGTH) 
test_features = convert_EHRexamples_to_features(test_f, MAX_SEQ_LENGTH)
test_features2 = convert_EHRexamples_to_features(test_f, MAX_SEQ_LENGTH)
valid_features = convert_EHRexamples_to_features(valid_f, MAX_SEQ_LENGTH)
train = BERTdataEHR(train_features)
test = BERTdataEHR(test_features)
test2 = BERTdataEHR(test_features2)
valid = BERTdataEHR(valid_features)
print (' creating the list of training minibatches')
train_mbs = list(BERTdataEHRloader(train, batch_size = BATCH_SIZE))
print (' creating the list of test minibatches')
test_mbs = list(BERTdataEHRloader(test, batch_size = BATCH_SIZE))
print (' creating the list of test2 minibatches')
test_mbs2 = list(BERTdataEHRloader(test2, batch_size = BATCH_SIZE))
print (' creating the list of valid minibatches')
valid_mbs = list(BERTdataEHRloader(valid, batch_size = BATCH_SIZE))

for run in range(8):### to average the results on 10 runs
    for model_type in ['Bert only']:              
            ehr_model = EHR_BERT_LR(input_size=20000, embed_dim=192, hidden_size=192) 
            ct = 0
            if run < 2:
                freeze = [0, 1]
            elif run < 4:
                freeze = [2, 3]
            elif run < 6:
                freeze = [4, 5]
            else:
                freeze = []  
            for param in ehr_model.parameters():
                if ct in freeze:
                    # print(freeze)
                    # for param in ch.parameters():
                    param.requires_grad = False
                ct += 1

            if use_cuda:
                ehr_model.cuda()
            optimizer = optim.Adam(ehr_model.parameters(), lr=LEARNING_RATE)
            out_dir_name='./fineTuneModels/'#+ str(i)
            trauc,vauc,testauc1,testauc2,bep=epochs_run(500,train = train_mbs, 
                                  valid = valid_mbs, 
                                  test1 = test_mbs,test2=test_mbs2, 
                                  model = ehr_model, 
                                  optimizer = optimizer,
                                  shuffle = True, 
                                  #batch_size = args.batch_size, 
                                  patience = 20,
                                  output_dir = out_dir_name,
                                  model_prefix = '{}_run_'.format(run))
            results.append([model_type,run,len(train_features),len(test_features),len(valid_features),trauc,vauc,testauc1,testauc2,bep])


Some weights of the model checkpoint at ./pytorch_model/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 

 creating the list of training minibatches
 creating the list of test minibatches
 creating the list of test2 minibatches
 creating the list of valid minibatches
0

 Epoch (0): Train_auc (0.5041184413580247), Valid_auc (0.4755756578947368) ,Training Average_loss (0.698458751042684), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.48335732984293195) ,Test_AUC2 (0.5358704188481676) , Test_eval_time (0m 0s) 
1

 Epoch (1): Train_auc (0.5918666747888239), Valid_auc (0.578125) ,Training Average_loss (0.6702477276325225), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.5770942408376962) ,Test_AUC2 (0.6016230366492146) , Test_eval_time (0m 0s) 
2

 Epoch (2): Train_auc (0.6490375243664717), Valid_auc (0.6580592105263158) ,Training Average_loss (0.6523031890392302), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.6315968586387435) ,Test_AUC2 (0.6383311518324608) , Test_eval_time (0m 0s) 
3

 Epoch (3): Train_auc (0.7062895955165692), Valid_auc (0.7268914473684212) ,Training Aver

Some weights of the model checkpoint at ./pytorch_model/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 


 Epoch (271): Train_auc (0.9994669834307992), Valid_auc (0.9810855263157895) ,Training Average_loss (0.03482029878068715), Train_time (0m 3s), Eval_time (0m 1s)
BestValidAuc 0.981497 has a TestAuc of 0.978979 at epoch 250 
0

 Epoch (0): Train_auc (0.5661062378167642), Valid_auc (0.6013157894736842) ,Training Average_loss (0.6800525069236755), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.5519240837696335) ,Test_AUC2 (0.5514005235602095) , Test_eval_time (0m 0s) 
1

 Epoch (1): Train_auc (0.6163966049382716), Valid_auc (0.6562499999999999) ,Training Average_loss (0.6437791148821513), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.6088612565445026) ,Test_AUC2 (0.5986780104712042) , Test_eval_time (0m 0s) 
2

 Epoch (2): Train_auc (0.6634112045159194), Valid_auc (0.6934210526315789) ,Training Average_loss (0.6470970213413239), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.6544764397905759) ,Test_AUC2 (0.6347120418848167) , Test_eval_time (0m 0s) 
3

 Epoch (3): Train

Some weights of the model checkpoint at ./pytorch_model/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 

...


 Epoch (61): Train_auc (0.9938190383365821), Valid_auc (0.9159539473684211) ,Training Average_loss (0.06851971400901676), Train_time (0m 3s), Eval_time (0m 1s)
BestValidAuc 0.936184 has a TestAuc of 0.924424 at epoch 40 
0

 Epoch (0): Train_auc (0.5142005766731643), Valid_auc (0.4273026315789473) ,Training Average_loss (0.6946251173814137), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.5256544502617801) ,Test_AUC2 (0.5446204188481675) , Test_eval_time (0m 0s) 
1

 Epoch (1): Train_auc (0.6100709673489279), Valid_auc (0.5338815789473684) ,Training Average_loss (0.6640688339869181), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.6080366492146596) ,Test_AUC2 (0.6007198952879581) , Test_eval_time (0m 0s) 
2

 Epoch (2): Train_auc (0.7376883325211177), Valid_auc (0.6879934210526316) ,Training Average_loss (0.6572432657082876), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.735589005235602) ,Test_AUC2 (0.7068848167539268) , Test_eval_time (0m 0s) 
3

 Epoch (3): Train_au

Some weights of the model checkpoint at ./pytorch_model/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 


 Epoch (64): Train_auc (0.9951531026640675), Valid_auc (0.9277960526315789) ,Training Average_loss (0.05334777773047487), Train_time (0m 3s), Eval_time (0m 1s)
BestValidAuc 0.941283 has a TestAuc of 0.926414 at epoch 43 
0

 Epoch (0): Train_auc (0.6115578297595842), Valid_auc (0.6666118421052631) ,Training Average_loss (0.6986834506193796), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.6348821989528797) ,Test_AUC2 (0.5806151832460733) , Test_eval_time (0m 0s) 
1

 Epoch (1): Train_auc (0.6766061565951917), Valid_auc (0.7404605263157894) ,Training Average_loss (0.664201537768046), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.700261780104712) ,Test_AUC2 (0.6397382198952879) , Test_eval_time (0m 0s) 
2

 Epoch (2): Train_auc (0.7194200779727096), Valid_auc (0.7732730263157894) ,Training Average_loss (0.6405810832977294), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.7391623036649214) ,Test_AUC2 (0.6756544502617801) , Test_eval_time (0m 0s) 
3

 Epoch (3): Train_auc

Some weights of the model checkpoint at ./pytorch_model/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 


 Epoch (64): Train_auc (0.9954302712800519), Valid_auc (0.9269736842105263) ,Training Average_loss (0.056477159851541135), Train_time (0m 3s), Eval_time (0m 1s)
BestValidAuc 0.937993 has a TestAuc of 0.924843 at epoch 43 
0

 Epoch (0): Train_auc (0.6029747400909682), Valid_auc (0.588404605263158) ,Training Average_loss (0.6855455080668132), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.5743062827225132) ,Test_AUC2 (0.45931937172774867) , Test_eval_time (0m 0s) 
1

 Epoch (1): Train_auc (0.6604532163742691), Valid_auc (0.6565789473684212) ,Training Average_loss (0.6506827592849731), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.6241492146596859) ,Test_AUC2 (0.5058246073298429) , Test_eval_time (0m 0s) 
2

 Epoch (2): Train_auc (0.7118152006172839), Valid_auc (0.7154605263157895) ,Training Average_loss (0.6494950572649637), Train_time (0m 3s), Eval_time (0m 1s)

 Test_AUC1 (0.6732068062827226) ,Test_AUC2 (0.5598429319371727) , Test_eval_time (0m 0s) 
3

 Epoch (3): Train_

In [10]:
df=pd.DataFrame(results)
df.columns=['Model','Run','Train_size','Test_size','Valid_size','Train_AUC','Valid_AUC','Test_AUC1','Test_AUC2','Best_Epoch']

In [11]:
df

,Model,Run,Train_size,Test_size,Valid_size,Train_AUC,Valid_AUC,Test_AUC1,Test_AUC2,Best_Epoch
0,Bert only,0,2124,591,236,0.998741,0.981497,0.978979,0.971505,250
1,Bert only,1,2124,591,236,0.997071,0.981990,0.979241,0.964647,209
2,Bert only,2,2124,591,236,0.977685,0.943421,0.923102,0.880419,42
3,Bert only,3,2124,591,236,0.978564,0.942845,0.927016,0.920458,40
4,Bert only,4,2124,591,236,0.972658,0.936184,0.924424,0.913940,40
5,Bert only,5,2124,591,236,0.979478,0.941283,0.926414,0.888809,43
6,Bert only,6,2124,591,236,0.979791,0.937993,0.924843,0.868390,43
7,Bert only,7,2124,591,236,0.977935,0.937171,0.927814,0.919411,42


In [15]:
df.assign(freeze_layer=["first_two", "first_two", "middle", "middle", "last_two", "last_two", "none", "none"])

,Model,Run,Train_size,Test_size,Valid_size,Train_AUC,Valid_AUC,Test_AUC1,Test_AUC2,Best_Epoch,freeze_layer
0,Bert only,0,2124,591,236,0.998741,0.981497,0.978979,0.971505,250,first_two
1,Bert only,1,2124,591,236,0.997071,0.981990,0.979241,0.964647,209,first_two
2,Bert only,2,2124,591,236,0.977685,0.943421,0.923102,0.880419,42,middle
3,Bert only,3,2124,591,236,0.978564,0.942845,0.927016,0.920458,40,middle
4,Bert only,4,2124,591,236,0.972658,0.936184,0.924424,0.913940,40,last_two
5,Bert only,5,2124,591,236,0.979478,0.941283,0.926414,0.888809,43,last_two
6,Bert only,6,2124,591,236,0.979791,0.937993,0.924843,0.868390,43,none
7,Bert only,7,2124,591,236,0.977935,0.937171,0.927814,0.919411,42,none


In [12]:
#df.to_csv('DHF_RNN_multirun_shuffled_1.csv')

In [13]:
desc2=df[['Model','Train_size','Test_AUC1']].groupby(['Model','Train_size']).describe()
desc3=df[['Model','Train_size','Test_AUC2']].groupby(['Model','Train_size']).describe()

In [14]:
desc3

Test_AUC2                                         \
                         count      mean       std      min       25%   
Model     Train_size                                                    
Bert only 2124             8.0  0.915947  0.037331  0.86839  0.886711   

                                                    
                           50%       75%       max  
Model     Train_size                                
Bert only 2124        0.916675  0.931505  0.971505